In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

#img1 = cv.imread('db/juice_train0.jpg',0)
img1 = cv.imread('db/license_plate.jpg',0)
# Initiate ORB detector
orb = cv.ORB_create()
# find the keypoints with ORB
kp1 = orb.detect(img1,None)

# compute the descriptors with ORB
kp1, des1 = orb.compute(img1, kp1)

    


def show_webcam(mirror=False):
    cam = cv.VideoCapture(0)
    while True:
        img=None
        ret_val, img2 = cam.read()
        if mirror: 
            img2 = cv.flip(img2, 1)
        #cv.imshow('my webcam', img2)
        kp2 = orb.detect(img2,None)
        kp2, des2 = orb.compute(img2, kp2)
        
        if cv.waitKey(1) == 27: 
            break  # esc to quit
        
        # create BFMatcher object
        bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)

        # Match descriptors.
        matches = bf.match(des1,des2)

        # Sort them in the order of their distance.
        matches = sorted(matches, key = lambda x:x.distance)

        # Draw first 10 matches.
        img3 = cv.drawMatches(img1,kp1,img2,kp2,matches[:15],None, flags=2)
        cv.imshow('my webcam', img3)
        #plt.imshow(img3),plt.show()
    cv.destroyAllWindows()

def main():
    show_webcam(mirror=False)

if __name__ == '__main__':
    main()

In [7]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

db=[]
for i in range(0,7):
    #img1 = cv.imread('db/juice_train0.jpg',0)
    img1 = cv.imread('db/img'+str(i)+'.jpg',0)
    # Initiate ORB detector
    orb = cv.ORB_create()
    # find the keypoints with ORB
    kp1 = orb.detect(img1,None)

    # compute the descriptors with ORB
    kp1, des1 = orb.compute(img1, kp1)
    db.append([img1,kp1,des1])


def show_webcam(mirror=False):
    cam = cv.VideoCapture(0)
    while True:
        img=None
        ret_val, img2 = cam.read()
        if mirror: 
            img2 = cv.flip(img2, 1)
        #cv.imshow('my webcam', img2)
        kp2 = orb.detect(img2,None)
        kp2, des2 = orb.compute(img2, kp2)
        
        if cv.waitKey(1) == 27: 
            break  # esc to quit
        
        # create BFMatcher object
        bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)

        # Match descriptors.
        best_idx=0
        best_matches=0
        for i in range(0,len(db)):
            matches = bf.match(db[i][2],des2)
            if len(matches)>best_matches:
                best_idx=i
                best_matches=len(matches)
        
        matches = bf.match(db[best_idx][2],des2)
        #print(len(matches))
        if(len(matches)>235):
            # Sort them in the order of their distance.
            matches = sorted(matches, key = lambda x:x.distance)

            # Draw first 10 matches.
            img3 = cv.drawMatches(db[best_idx][0],db[best_idx][1],img2,kp2,matches[:15],None, flags=2)
            cv.imshow('my webcam', img3)
            #plt.imshow(img3),plt.show()
        else:
            cv.imshow('my webcam', img2)
    cv.destroyAllWindows()

def main():
    show_webcam(mirror=False)

if __name__ == '__main__':
    main()

In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

#img1 = cv.imread('db/juice_train0.jpg',0)
img1 = cv.imread('db/license_plate.jpg',0)
# Initiate ORB detector
orb = cv.ORB_create()
# find the keypoints with ORB
kp1 = orb.detect(img1,None)

# compute the descriptors with ORB
kp1, des1 = orb.compute(img1, kp1)

def show_webcam(mirror=False):
    cam = cv.VideoCapture(0)
    while True:
        img=None
        ret_val, img2 = cam.read()
        if mirror: 
            img2 = cv.flip(img2, 1)
        #cv.imshow('my webcam', img2)
        kp2 = orb.detect(img2,None)
        kp2, des2 = orb.compute(img2, kp2)
        
        if cv.waitKey(1) == 27: 
            break  # esc to quit
        
        
        FLANN_INDEX_KDTREE = 0
        #index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        index_params= dict(algorithm = 6,
                   table_number = 6, # 12
                   key_size = 12,     # 20
                   multi_probe_level = 1) #2
        search_params = dict(checks=50)   # or pass empty dictionary

        flann = cv.FlannBasedMatcher(index_params,search_params)

        matches = flann.knnMatch(des1,des2,k=2)
        
        print("--------------------")
        print(matches[2][0])
        print("--------------------")

        # Need to draw only good matches, so create a mask
        matchesMask = [[0,0] for i in range(len(matches))]

        # ratio test as per Lowe's paper
        #for i,(m,n) in enumerate(matches):
        #    print(m)
        #    print(n)
        #    if m.distance < 0.7*n.distance:
        #       matchesMask[i]=[1,0]
        
        print("pase")
        print(len(matches))
        for i in range(len(matches)):
            #print(m)
            print(i)
            if matches[i][0].distance < 0.7*matches[i][1].distance:
                #print("i")
                matchesMask[i]=[1,0]

        draw_params = dict(matchColor = (0,255,0),
                           singlePointColor = (255,0,0),
                           matchesMask = matchesMask,
                           flags = 0)

        img3 = cv.drawMatchesKnn(img1,kp1,img2,kp2,matches,None,**draw_params)
        
        cv.imshow('my webcam', img3)
        #plt.imshow(img3,),plt.show()
        
    cv.destroyAllWindows()

def main():
    show_webcam(mirror=False)

if __name__ == '__main__':
    main()